In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import os
import glob
from os import walk
from matplotlib.gridspec import GridSpec
from IPython.display import display, clear_output
from nano_lab import experiments
from mpl_toolkits.axes_grid1 import make_axes_locatable,ImageGrid
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('../src/plotstyle.mplstyle')
exp= experiments('nano-lab','nsom','fg163r',False)
exp.dframe



,No. Dir,Name Dir
0,0,2022-10-20-nsom.h5
1,1,2022-10-23-nsom.h5
2,2,2022-10-25-nsom.h5


In [3]:
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation
import math
import warnings
warnings.filterwarnings("ignore")


class afm_nsom:
    def __init__(self,alldata,measure):
        self.alldata=alldata
        self.measure = measure
        self.afm_nsom_data = self.alldata[self.measure][0]
        self.xi=0;self.xf=self.afm_nsom_data.shape[1]
        self.yi=0;self.yf=self.afm_nsom_data.shape[0]
        self.afm         = self.afm_nsom_data[:,:,0]
        self.lockin      =  self.afm_nsom_data[:,:,1]
        self.multimeter  =  self.afm_nsom_data[:,:,2]


        try:
            self.amplitud = self.alldata[self.measure][1]
            self.fase = self.alldata[self.measure][2]
        except:
            pass




In [52]:

exptoanalysis =afm_nsom(exp.data,2)
afm    = exptoanalysis.afm
lockin =  exptoanalysis.lockin
multimeter =  exptoanalysis.multimeter
nx, ny = afm.shape[1], afm.shape[0]
x, y = np.meshgrid(np.arange(0, nx, 1), np.arange(0, ny, 1))


fig = make_subplots(
        rows=1, cols=3,
        specs=[[{'type': 'surface'}, {'type': 'surface'},{'type': 'surface'}]],print_grid=False, shared_xaxes=True,shared_yaxes=True,)

    # Generate data
nx = afm.shape[0]
ny = afm.shape[1]
x = np.arange(0, nx, 1)
y = np.arange(0, ny, 1)
xt =np.arange(x[0],x[-1],7)

exp2plots = [afm,lockin,multimeter]
for i,expriment in enumerate(exp2plots):
    fig.add_trace(
        go.Surface( z=expriment, colorscale='Plotly3',showscale=False, reversescale=True,),row=1, col=i+1)
    fig.update_traces(contours_x=dict(show=True, usecolormap=True, project_x=True))
    fig.update_traces(contours_y=dict(show=True, usecolormap=True, project_y=True))


fig.update_layout(
     template="plotly_dark",
     width=1200,
    font=dict(
        family="Computer Modern Roman,serif",
        color='white',
        size=13,
    ),
    scene=dict(
    xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",),
    yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",)),
    scene2=dict(
    xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",),
    yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",)),
    scene3=dict(
    xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",),
    yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",)),
 )

fig.show()